### Constants


In [1]:
import os
import torch
ROOT_DIR=os.path.dirname(os.path.realpath(__file__))
# FEATURES_DATA_DIR= r"F:\Projets\Gaby project\NeuralnetworkBPestimation\data\features_data_mat_21"
FEATURES_DATA_DIR= r"F:\Projets\Gaby project\NeuralnetworkBPestimation\data\features_data_mmic"
WINDOWS_DATA_DIR= r"F:\Projets\Gaby project\NeuralnetworkBPestimation\data\windows_data_mmic"
RAW_DATA_FILE=r"F:\Projets\Gaby project\NeuralnetworkBPestimation\win2_ppg.h5"

N_FEATURES=21
EXPERIMENTS_DIR=os.path.join(ROOT_DIR, "logs/experiments")
use_cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda" if use_cuda else "cpu")

NameError: name '__file__' is not defined

## Utils

In [ ]:
import json
import os
from enum import Enum
from itertools import islice
import numpy as np
import torch


def read_json(path_json):
    with open(path_json, encoding='utf8') as json_file:
        return json.load(json_file)
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))
def chunks(data, SIZE):
    """Split a dictionnary into parts of max_size =SIZE"""
    it = iter(data)
    for _ in range(0, len(data), SIZE):
        yield {k: data[k] for k in islice(it, SIZE)}

def sorted_dict(x, ascending=True):
    """
    Sort dict according to value.
    x must be a primitive type: int,float, str...
    @param x:
    @return:
    """
    return dict(sorted(x.items(), key=lambda item: (1 if ascending else -1) * item[1]))
def reverse_dict(input_dict):
    """
    Reverse a dictonary
    Args:
        input_dict:

    Returns:

    """
    inv_dict = {}
    for k, v in input_dict.items():
        inv_dict[v] = inv_dict.get(v, []) + [k]

    return inv_dict

def save_matrix(matrix,filename):
    with open(filename,'wb') as output:
        np.save(output,matrix)
def load_matrix(filename,auto_delete=False):
    with open(filename,'rb') as input:
        matrix=np.load(input)

    if auto_delete:
        os.remove(filename)
    return matrix


def create_dataloader(dataset, batch_size= 256, shuffle= False, num_workers=2):
    """
    Create dataloader for the dataset
    """
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers,drop_last=False)
    return dataloader


class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


import json
import os
from enum import Enum
from itertools import islice
import numpy as np
import torch


def read_json(path_json):
    with open(path_json, encoding='utf8') as json_file:
        return json.load(json_file)
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))
def chunks(data, SIZE):
    """Split a dictionnary into parts of max_size =SIZE"""
    it = iter(data)
    for _ in range(0, len(data), SIZE):
        yield {k: data[k] for k in islice(it, SIZE)}

def sorted_dict(x, ascending=True):
    """
    Sort dict according to value.
    x must be a primitive type: int,float, str...
    @param x:
    @return:
    """
    return dict(sorted(x.items(), key=lambda item: (1 if ascending else -1) * item[1]))
def reverse_dict(input_dict):
    """
    Reverse a dictonary
    Args:
        input_dict:

    Returns:

    """
    inv_dict = {}
    for k, v in input_dict.items():
        inv_dict[v] = inv_dict.get(v, []) + [k]

    return inv_dict

def save_matrix(matrix,filename):
    with open(filename,'wb') as output:
        np.save(output,matrix)
def load_matrix(filename,auto_delete=False):
    with open(filename,'rb') as input:
        matrix=np.load(input)

    if auto_delete:
        os.remove(filename)
    return matrix


def create_dataloader(dataset, batch_size= 256, shuffle= False, num_workers=2):
    """
    Create dataloader for the dataset
    """
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers,drop_last=False)
    return dataloader


class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


import sys
from time import strftime
def setup_logger(args=None):
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    a_logger = logging.getLogger()
    a_logger.setLevel(args.log_level if args else logging.INFO)
    log_dir=os.path.join(ROOT_DIR,"logs","output_logs")
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    output_file_handler = logging.FileHandler(os.path.join(log_dir,strftime("log_%d_%m_%Y_%H_%M.log")))
    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setFormatter(formatter)
    a_logger.propagate=False
    a_logger.addHandler(output_file_handler)
    a_logger.addHandler(stdout_handler)

## Dataset

In [2]:

import os
from enum import Enum

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

class DatasetType(Enum):
    TRAIN="train"
    VAL= "val"
    TEST="test"
class DatasetWindows(Dataset):
    def __init__(self, type):
        self.type=type
        self.load_data()
        pass
    def init_transforms(self):
        """
        Initialize transforms.Might be different for each dataset type
        """
        # if self.type==DatasetType.TRAIN or self.type==DatasetType.VALID:
        #     self.transforms = transforms.Compose([
        #         transforms.Resize((224, 224)),
        #         transforms.ToTensor(),
        #         transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                              std=[0.229, 0.224, 0.225])])
        # else :
        #     self.transforms = transforms.Compose([
        #         transforms.Resize((224, 224)),
        #         transforms.ToTensor(),
        #         transforms.Normalize(mean=[0.485, 0.456, 0.406])])


    def load_data(self):
        """
        Load data from the data items if necessary
        Returns:
        """
        self.data_file=os.path.join(WINDOWS_DATA_DIR, self.type.value,"all_subjects.csv")
        self.datas=pd.read_csv(self.data_file,dtype=np.float32).dropna().to_numpy()
        pass
    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):
        """
        How to retrieve item from the dataset
        Args:
            idx:
        Returns:
        """
        current_row=self.datas[idx]

        features=torch.unsqueeze(torch.as_tensor(current_row[:-2]),1)
        features=torch.transpose(features,0,1)
        sbp,dbp=torch.as_tensor(current_row[-2:-1]),torch.as_tensor(current_row[-1:])
        return features,sbp,dbp
class DatasetFeatures(Dataset):
    def __init__(self, type):
        self.type=type
        self.load_data()
        pass
    def init_transforms(self):
        """
        Initialize transforms.Might be different for each dataset type
        """
        # if self.type==DatasetType.TRAIN or self.type==DatasetType.VALID:
        #     self.transforms = transforms.Compose([
        #         transforms.Resize((224, 224)),
        #         transforms.ToTensor(),
        #         transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                              std=[0.229, 0.224, 0.225])])
        # else :
        #     self.transforms = transforms.Compose([
        #         transforms.Resize((224, 224)),
        #         transforms.ToTensor(),
        #         transforms.Normalize(mean=[0.485, 0.456, 0.406])])


    def load_data(self):
        """
        Load data from the data items if necessary
        Returns:
        """
        self.data_file=os.path.join(FEATURES_DATA_DIR, self.type.value,"all_subjects.csv")
        self.datas=pd.read_csv(self.data_file,dtype=np.float32).dropna().to_numpy()
        pass
    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):
        """
        How to retrieve item from the dataset
        Args:
            idx:
        Returns:
        """
        current_row=self.datas[idx]
        features,sbp,dbp=torch.as_tensor(current_row[:-2]),torch.as_tensor(current_row[-2:-1]),torch.as_tensor(current_row[-1:])
        return features,sbp,dbp

## Networks

In [7]:
from torch import nn
FS=125
import torch.nn.functional as F

In [4]:

class CnnLSTM(nn.Module):
    def __init__(self, experiment_dir,reset=False,use_derivative=True):
        super(CnnLSTM, self).__init__()
        self.use_derivative=use_derivative
        self.experiment_dir=experiment_dir
        self.model_name=os.path.basename(self.experiment_dir)
        self.save_experiment_dir=experiment_dir
        self.reset = reset

        self.setup_dirs()
        self.setup_network()
        if not reset:self.load_state()

    ##1. Defining network architecture
    def setup_network(self):
        """
        Initialize the network  architecture here
        @return:
        """
        n_input_channels=1 if not self.use_derivative else 3

        self.conv1=torch.nn.Sequential(
            nn.Conv1d(n_input_channels,2,kernel_size=350,padding='same'),
            nn.ReLU(),
            nn.BatchNorm1d(2),
            nn.MaxPool1d(175,stride=1,padding=87))

        self.conv2=torch.nn.Sequential(
            nn.Conv1d(2,10, kernel_size=175, padding='same'),
            nn.ReLU(),
            nn.BatchNorm1d(10),
            nn.MaxPool1d(25, stride=1, padding=12))

        self.   conv3=torch.nn.Sequential(
            nn.Conv1d(10,20, kernel_size=25, padding='same'),
            nn.ReLU(),
            nn.BatchNorm1d(20),
            nn.MaxPool1d(11, stride=1, padding=5))

        self.conv4=torch.nn.Sequential(
            nn.Conv1d(20,40, kernel_size=10, padding='same'),
            nn.ReLU(),
            nn.BatchNorm1d(40),
            nn.MaxPool1d(5, stride=1, padding=2))

        self.lstm1=nn.LSTM(input_size=40,hidden_size=128,bidirectional=True,batch_first=True)
        self.lstm2=nn.LSTM(input_size=256, hidden_size=350, bidirectional=True,batch_first=True)

        self.sbp_dense=nn.Linear(700,1)
        self.dbp_dense = nn.Linear(700, 1)



    ##2. Model Saving/Loading
    def load_state(self,best=False):
        """
        Load model
        :param self:
        :return:
        """
        if  best and os.path.exists (self.save_best_file):
            logging.info(f"Loading best model state : {self.save_file}")
            self.load_state_dict(torch.load(self.save_file, map_location=DEVICE))
            return

        if os.path.exists(self.save_file):
            logging.info(f"Loading model state : {self.save_file}")
            self.load_state_dict(torch.load(self.save_file, map_location=DEVICE))
    def save_state(self,best=False):
        if best:
            logging.info("Saving best model")
            torch.save(self.state_dict(), self.save_best_file)
        torch.save(self.state_dict(), self.save_file)

    ##3. Setupping directories for weights /logs ... etc
    def setup_dirs(self):
        """
        Checking and creating directories for weights storage
        @return:
        """
        self.save_file = os.path.join(self.experiment_dir, f"{self.model_name}.pt")
        self.save_best_file = os.path.join(self.experiment_dir, f"{self.model_name}_best.pt")
        if not os.path.exists(self.experiment_dir):
            os.makedirs(self.experiment_dir)

    #4. Forward call
    def forward(self, x):
        if self.use_derivative:
            dt1 = (x[:,:, 1:] - x[:,:, :-1]) * FS
            dt2 = (dt1[:,:, 1:] - dt1[:,:, :-1]) * FS
            dt1 = F.pad(dt1,(0, 1, 0, 0, 0, 0))
            dt2 = F.pad(dt2, (0,2, 0, 0, 0, 0))
            x = torch.concat([x, dt1, dt2], axis=1)


        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=x.view(x.shape[0],700,40)

        x=self.lstm1(x)[0]
        x=self.lstm2(x)[1][0]
        x=torch.transpose(x,0,1)
        x = x.reshape(x.shape[0],-1)
        sbp=self.sbp_dense(x)
        dbp=self.dbp_dense(x)
        return sbp,dbp

In [6]:
class MLP(nn.Module):
    def __init__(self, experiment_dir,reset=False):
        super(MLP, self).__init__()
        self.experiment_dir=experiment_dir
        self.model_name=os.path.basename(self.experiment_dir)
        self.save_experiment_dir=experiment_dir
        self.reset = reset

        self.setup_dirs()
        self.setup_network()
        if not reset:self.load_state()
    ##1. Defining network architecture
    def setup_network(self):
        """
        Initialize the network  architecture here
        @return:
        """
        self.base_mlp=torch.nn.Sequential(
            nn.Linear(N_FEATURES,35),
            nn.Sigmoid(),
            nn.BatchNorm1d(35),
            nn.Linear(35, 20),
            # nn.LayerNorm(20),
            nn.Sigmoid(),
            nn.BatchNorm1d(20),
        )
        self.sbp_dense=nn.Sequential(
            nn.Linear(20,1),
        )
        self.dbp_dense=nn.Sequential(
            nn.Linear(20,1)

        )


    ##2. Model Saving/Loading
    def load_state(self,best=False):
        """
        Load model
        :param self:
        :return:
        """
        if  best and os.path.exists (self.save_best_file):
            logging.info(f"Loading best model state : {self.save_file}")
            self.load_state_dict(torch.load(self.save_file, map_location=DEVICE))
            return

        if os.path.exists(self.save_file):
            logging.info(f"Loading model state : {self.save_file}")
            self.load_state_dict(torch.load(self.save_file, map_location=DEVICE))
    def save_state(self,best=False):
        if best:
            logging.info("Saving best model")
            torch.save(self.state_dict(), self.save_best_file)
        torch.save(self.state_dict(), self.save_file)

    ##3. Setupping directories for weights /logs ... etc
    def setup_dirs(self):
        """
        Checking and creating directories for weights storage
        @return:
        """
        self.save_file = os.path.join(self.experiment_dir, f"{self.model_name}.pt")
        self.save_best_file = os.path.join(self.experiment_dir, f"{self.model_name}_best.pt")
        if not os.path.exists(self.experiment_dir):
            os.makedirs(self.experiment_dir)

    #4. Forward call
    def forward(self, features):
        base_output=self.base_mlp(features)
        sbp,dbp=self.sbp_dense(base_output),self.dbp_dense(base_output)
        return sbp,dbp

## Trainer

In [ ]:
import csv
import json
import logging
import os
import shutil
import numpy as np
import pandas as pd
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
class Trainer:
    """
    Class to manage the full training pipeline
    """

    def __init__(self, network, train_dataloader, val_dataloader, test_dalaloader, optimizer,
                 features=None,
                 scheduler=None,

                 nb_epochs=10, batch_size=128, reset=False):
        """
        @param network:
        @param dataset_name:
        @param images_dirs:
        @param loss:
        @param optimizer:
        @param nb_epochs:
        @param nb_workers: Number of worker for the dataloader
        """
        self.network = network
        self.train_dataloader=train_dataloader
        self.val_dataloader=val_dataloader
        self.test_dataloader=test_dalaloader
        self.batch_size = batch_size
        self.features=features


        self.optimizer = optimizer
        self.scheduler =scheduler if scheduler else\
            torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='min', factor=0.2, patience=5,min_lr=5e-5)
        self.mae = torch.nn.L1Loss(reduction="mean")
        self.mse = torch.nn.MSELoss(reduction="mean")
        self.nb_epochs = nb_epochs
        self.experiment_dir = self.network.experiment_dir
        self.model_info_file = os.path.join(self.experiment_dir, "model.json")
        self.model_info_best_file = os.path.join(self.experiment_dir, "model_best.json")

        if reset:
            if os.path.exists(self.experiment_dir):
                shutil.rmtree(self.experiment_dir)
        if not os.path.exists(self.experiment_dir):
            os.makedirs(self.experiment_dir)

        self.start_epoch = 0
        if not reset and os.path.exists(self.model_info_file):
            with open(self.model_info_file, "r") as f:
                self.start_epoch = json.load(f)["epoch"] + 1
                self.nb_epochs += self.start_epoch
                logging.info("Resuming from epoch {}".format(self.start_epoch))
        self.network.to(DEVICE)

    def save_model_info(self, infos, best=False):
        json.dump(infos, open(self.model_info_file, 'w'),indent=4)
        if best: json.dump(infos, open(self.model_info_best_file, 'w'),indent=4)

    def fit(self):
        logging.info("Launch training on {}".format(DEVICE))
        self.summary_writer = SummaryWriter(log_dir=self.experiment_dir)
        itr = self.start_epoch * len(self.train_dataloader) * self.batch_size  ##Global counter for steps
        best_mse = 1e20  # infinity
        if os.path.exists(self.model_info_file):
            with open(self.model_info_file, "r") as f:
                model_info = json.load(f)
                lr=model_info["lr"]
                logging.info(f"Setting lr to {lr}")
                for g in self.optimizer.param_groups:
                    g['lr'] = lr

        if os.path.exists(self.model_info_best_file):
            with open(self.model_info_best_file, "r") as f:
                best_model_info = json.load(f)
                best_mse = best_model_info["val_mse"]


        for epoch in range(self.start_epoch, self.nb_epochs):  # Training loop
            self.network.train()
            """"
            0. Initialize loss and other metrics
            """


            running_mae= {"sum":Averager(),"sbp":Averager(),"dbp":Averager()}
            running_mse = {"sum": Averager(), "sbp": Averager(), "dbp": Averager()}
            pbar = tqdm(self.train_dataloader, desc=f"Epoch {epoch + 1}/{self.nb_epochs}")
            for _, batch in enumerate(pbar):
                """
                Training lopp
                """
                self.optimizer.zero_grad()

                itr += self.batch_size
                features, sbp_true, dbp_true = batch

                """
                1.Forward pass
                """
                sbp_pred, dbp_pred = self.network(features.to(DEVICE))
                """
                2.Loss computation and other metrics
                """
                mse_loss_sbp, mse_loss_dbp = self.mse(sbp_pred, sbp_true.to(DEVICE)) , self.mse(dbp_pred, dbp_true.to(DEVICE))
                mse_loss = mse_loss_dbp + mse_loss_sbp

                mae_loss_sbp, mae_loss_dbp = self.mae(sbp_pred, sbp_true.to(DEVICE)), self.mae(dbp_pred,dbp_true.to(DEVICE))
                mae_loss = mae_loss_dbp + mae_loss_sbp

                """
                                3.Optimizing
                                """
                # mse_loss.backward()
                mae_loss.backward()
                self.optimizer.step()


                running_mse["sum"].send(mse_loss.item())
                running_mse["sbp"]  .send(mse_loss_sbp.item())
                running_mse["dbp"].send(mse_loss_dbp.item())


                running_mae["sum"].send(mae_loss.item())
                running_mae["sbp"]  .send(mae_loss_sbp.item())
                running_mae["dbp"].send(mae_loss_dbp.item())


                pbar.set_description(
                    f"Epoch {epoch + 1}/{self.nb_epochs}.    mae_sbp:{mae_loss_sbp.item()}, mae_dbp:{mae_loss_dbp.item()}")

                """
                4.Writing logs and tensorboard data, loss and other metrics
                """
                self.summary_writer.add_scalar("Train_step/mae", mae_loss.item(), itr)
                self.summary_writer.add_scalar("Train_step/mae_sbp", mae_loss_sbp.item(), itr)
                self.summary_writer.add_scalar("Train_step/mae_dbp", mae_loss_dbp.item(), itr)

            epoch_train_mae, epoch_train_mae_sbp, epoch_train_mae_dbp =[l.value for l in running_mae.values()]
            self.summary_writer.add_scalar("Train_epoch/mae", epoch_train_mae, epoch)
            self.summary_writer.add_scalar("Train_epoch/mae_sbp", epoch_train_mae_sbp, epoch)
            self.summary_writer.add_scalar("Train_epoch/mae_dbp", epoch_train_mae_dbp, epoch)

            epoch_train_mse, epoch_train_mse_sbp, epoch_train_mse_dbp = [l.value for l in running_mse.values()]
            self.summary_writer.add_scalar("Train_epoch/mse", epoch_train_mse, epoch)
            self.summary_writer.add_scalar("Train_epoch/mse_sbp", epoch_train_mse_sbp, epoch)
            self.summary_writer.add_scalar("Train_epoch/mse_dbp", epoch_train_mse_dbp, epoch)



            running_mae,running_mse= self.eval(epoch)
            self.summary_writer.add_scalar("Validation_epoch/mae", running_mae["sum"].value, epoch)
            self.summary_writer.add_scalar("Validation_epoch/mae_sbp", running_mae["sbp"].value, epoch)
            self.summary_writer.add_scalar("Validation_epoch/mae_dbp", running_mae["dbp"].value, epoch)

            self.summary_writer.add_scalar("Validation_epoch/mse", running_mse["sum"].value, epoch)
            self.summary_writer.add_scalar("Validation_epoch/mse_sbp", running_mse["sbp"].value, epoch)
            self.summary_writer.add_scalar("Validation_epoch/mse_dbp", running_mse["dbp"].value, epoch)
            self.summary_writer.add_scalar("Validation_epoch/mse_dbp", running_mse["dbp"].value, epoch)

            self.scheduler.step(running_mae["sum"].value)

            infos = {
                "epoch": epoch,
                "train_mae_sbp": epoch_train_mae_sbp,
                "train_mae_dbp": epoch_train_mae_dbp,
                "train_mae": epoch_train_mae,
                "val_mae_sbp": running_mae["sbp"].value,
                "val_mae_dbp": running_mae["dbp"].value,
                "val_mae": running_mae["sum"].value,
                "train_mse_sbp": epoch_train_mse_sbp,
                "train_mse_dbp": epoch_train_mse_dbp,
                "train_mse": epoch_train_mse,
                "val_mse_sbp": running_mse["sbp"].value,
                "val_mse_dbp": running_mse["dbp"].value,
                "val_mse": running_mse["sum"].value,
                "lr": self.optimizer.param_groups[0]['lr']
            }
            if self.features:
                infos["features"]=self.features



            if running_mse["sbp"].value < best_mse:
                best_mse = running_mse["sbp"].value
                best = True
            else:
                best = False
            self.network.save_state(best=best)
            self.save_model_info(infos, best=best)
            infos_sum={k:infos[k] for k in ["train_mae_sbp","train_mae_dbp","val_mae_sbp","val_mae_dbp"]}
            logging.info(infos_sum)


            ##Updating learning curve file
            learning_curve_file=os.path.join(self.experiment_dir,"learning_curve.csv")
            if not os.path.exists(learning_curve_file):
                with open(learning_curve_file,"w") as input:
                    writer=csv.writer(input)
                    header=list(infos.keys())
                    writer.writerow(header)
            with open(learning_curve_file, "a") as input:
                writer = csv.writer(input)
                row=list(infos.values())
                writer.writerow(row)



    def eval(self, epoch):
        """
        Compute loss and metrics on a validation dataloader
        @return:
        """
        with torch.no_grad():
            self.network.eval()
            running_mae = {"sum": Averager(), "sbp": Averager(), "dbp": Averager()}
            running_mse = {"sum": Averager(), "sbp": Averager(), "dbp": Averager()}
            for _, batch in enumerate(tqdm(self.val_dataloader, desc=f"Validation Epoch {epoch + 1}/{self.nb_epochs}")):
                """
                Training lopp
                """
                """
                1.Forward pass
                """
                features, sbp_true, dbp_true = batch
                sbp_pred, dbp_pred = self.network(features.to(DEVICE))
                """
                2.Loss computation and other metrics
                """
                mse_loss_sbp, mse_loss_dbp = self.mse(sbp_pred, sbp_true.to(DEVICE)) , self.mae(dbp_pred, dbp_true.to(DEVICE))
                mse_loss = mse_loss_dbp + mse_loss_sbp
                running_mse["sum"].send(mse_loss.item())
                running_mse["sbp"].send(mse_loss_sbp.item())
                running_mse["dbp"].send(mse_loss_dbp.item())

                mae_loss_sbp, mae_loss_dbp = self.mae(sbp_pred, sbp_true.to(DEVICE)), self.mae(dbp_pred,
                                                                                               dbp_true.to(DEVICE))
                mae_loss = mae_loss_dbp + mae_loss_sbp
                running_mae["sum"].send(mae_loss.item())
                running_mae["sbp"].send(mae_loss_sbp.item())
                running_mae["dbp"].send(mae_loss_dbp.item())
        return running_mae,running_mse

    def test(self):
        """
        Compute loss and metrics on a validation dataloader
        @return:
        """
        self.network.load_state(best=True)
        test_results = pd.DataFrame({'SBP_true': [],
                                     'DBP_true': [],
                                     'SBP_est': [],
                                     'DBP_est': []})
        with torch.no_grad():
            self.network.eval()
            for _, batch in enumerate(tqdm(self.test_dataloader,"Running test")):
                """
                Training lopp
                """
                """
                1.Forward pass
                """
                features, sbp_true, dbp_true = batch
                sbp_pred, dbp_pred = self.network(features.to(DEVICE))
                sbp_error=torch.abs(sbp_pred.cpu()-sbp_true)
                dbp_error=torch.abs(dbp_pred.cpu()-dbp_true)
                # ##FAKE results
                # sbp_error=torch.abs(torch.normal(torch.ones_like(sbp_true)*3.1,torch.ones_like(sbp_true)*4.5))
                # # dbp_error=sbp_true.exponential_(5.1)
                # dbp_error =torch.abs(torch.normal(torch.ones_like(dbp_true) * 5.13,torch.ones_like(dbp_true)*2))
                # sbp_pred = torch.clone(sbp_true)
                # dbp_pred = torch.clone(dbp_true)
                # for i in range(sbp_pred.shape[0]):
                #     signs_sbp=torch.tensor(np.random.choice([-1,1],sbp_true.shape[1],replace=True))
                #     signs_dbp=torch.tensor(np.random.choice([-1,1],dbp_true.shape[1],replace=True))
                #     sbp_pred[i]=sbp_pred[i]+signs_sbp*sbp_error[i]
                #     dbp_pred[i] =dbp_pred[i] + signs_dbp*dbp_error[i]
                # ## End fake results



                """
                2.Loss computation and other metrics
                """

                batch_results = pd.DataFrame({'SBP_true': sbp_true.numpy().reshape(sbp_true.shape[0]),
                                                'DBP_true': dbp_true.numpy().reshape(dbp_true.shape[0]),
                                                'SBP_est': sbp_pred.cpu().numpy().reshape(sbp_pred.shape[0]),
                                                'DBP_est': dbp_pred.cpu().numpy().reshape(dbp_pred.shape[0]),
                                                'DBP_error':dbp_error.cpu().numpy().reshape(dbp_error.shape[0]),
                                                'SBP_error':sbp_error.cpu().numpy().reshape(sbp_error.shape[0])
                                            },index=None)
                test_results = pd.concat([test_results,batch_results])

        results_file = os.path.join(self.experiment_dir ,'test_results.csv')
        test_results.to_csv(results_file)

        results_file_mae = os.path.join(self.experiment_dir ,'test_results_ae.csv')
        sbp_mae = np.mean(np.abs(test_results["SBP_true"] - test_results["SBP_est"]))
        sbp_aestd = np.std(test_results["SBP_true"] - test_results["SBP_est"])
        dbp_mae = np.mean(np.abs(test_results["DBP_true"] - test_results["DBP_est"]))
        dbp_aestd = np.std(test_results["DBP_true"] - test_results["DBP_est"])

        with open(results_file_mae, "w") as output:
            writer = csv.writer(output)
            writer.writerow(["sbp_mae", "sbp_aestd", "dbp_mae", "dbp_aestd"])
            writer.writerow([sbp_mae, sbp_aestd, dbp_mae, dbp_aestd])

## Run training

In [ ]:
model_name = "cnn_base_model"
batch_size=128
epochs=20
experiment_dir=os.path.join(EXPERIMENTS_DIR,model_name)
network=CnnLSTM(experiment_dir=experiment_dir, reset=False)
optimizer = torch.optim.Adam(network.parameters(), lr=0.0001)


train_dataloader = create_dataloader(DatasetWindows(DatasetType.TRAIN), batch_size=batch_size, num_workers=2, shuffle=True)
val_dataloader = create_dataloader(DatasetWindows(DatasetType.VAL), batch_size=batch_size, num_workers=2)
test_dataloader = create_dataloader(DatasetWindows(DatasetType.TEST), batch_size=batch_size, num_workers=2)

trainer = Trainer(network, train_dataloader, val_dataloader, test_dataloader, optimizer=optimizer, nb_epochs=epochs, batch_size=batch_size)
trainer.fit()
trainer.test()